In [7]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
import numpy as np
import importlib
import sklearn
import random
import torch
import math

import socialSig
importlib.reload(socialSig)
from helpers import *

In [8]:
####### Load our Data
#y - 'number_moved'
#x - 'everything else that is or can be represented as a float.'
devSet = pd.read_csv("./us_migration.csv")
devSet = devSet.loc[:, ~devSet.columns.str.contains('^Unnamed')]
devSet = devSet.apply(lambda x: pd.to_numeric(x, errors='coerce'))
devSet = devSet.dropna(axis=1)
devSet = devSet.drop(['sending'], axis = 1)

print(devSet.head())

y = torch.Tensor(devSet['US_MIG_05_10'].values)
X = devSet.loc[:, devSet.columns != "US_MIG_05_10"].values

mMScale = preprocessing.MinMaxScaler()
X = mMScale.fit_transform(X)

   sending_citizen_unspecified  sending_citizenship_unknown  \
0                  1134.995083                   243.827586   
1                   917.067746                   600.000000   
2                   637.977315                   557.150000   
3                  1178.378744                  1678.562500   
4                  2231.128863                  2916.538705   

   sending_household_not_owned  sending_household_owned  \
0                  1178.209016              1106.832815   
1                  1274.160656               795.118020   
2                   729.454795               610.791414   
3                  1560.244328              1084.109969   
4                  2104.943216              2278.834178   

   sending_household_owned_unknown  sending_indigeneity  sending_internet  \
0                       736.562500           918.960526       2111.233685   
1                        -1.000000            -1.000000       1988.103175   
2                       468.392857 

In [9]:
len(X[0])

29

In [10]:
def train_test_split(X, y, split):

    train_num = int(len(X) * split)
    val_num = int(len(X) - train_num)

    train_indices = random.sample(range(len(X)), train_num)
    val_indices = [i for i in range(len(X)) if i not in train_indices]

    x_train, x_val = X[train_indices], X[val_indices]
    y_train, y_val = y[val_indices], y[val_indices]

    return x_train, y_train, x_val, y_val

In [11]:
####### Build and fit the Model
lr = 1e-6
batchSize = 50
model = socialSig.SocialSigNet(X=X, outDim = batchSize)
epochs = 1

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

In [12]:
x_train, y_train, x_val, y_val = train_test_split(X, y, .80)

train = [(k,v) for k,v in zip(x_train, y_train)]
val = [(k,v) for k,v in zip(x_val, y_val)]

train = torch.utils.data.DataLoader(train, batch_size = batchSize, shuffle = True)
val = torch.utils.data.DataLoader(val, batch_size = batchSize, shuffle = True)

In [14]:
for inputs, output in train:
    print(inputs)

tensor([[0.1750, 0.0627, 0.1885,  ..., 0.1726, 0.1140, 0.1119],
        [0.0618, 0.0012, 0.1161,  ..., 0.0564, 0.0093, 0.0778],
        [0.3315, 0.0000, 0.3878,  ..., 0.3056, 0.0834, 0.4247],
        ...,
        [0.2739, 0.0308, 0.2569,  ..., 0.2696, 0.1155, 0.2711],
        [0.2552, 0.1112, 0.2368,  ..., 0.2534, 0.0486, 0.1823],
        [0.1958, 0.0588, 0.2346,  ..., 0.1934, 0.1110, 0.1771]],
       dtype=torch.float64)
tensor([[0.1453, 0.0508, 0.1799,  ..., 0.1416, 0.0445, 0.1072],
        [0.0908, 0.0740, 0.1275,  ..., 0.0891, 0.0455, 0.0494],
        [0.1359, 0.0618, 0.1424,  ..., 0.1330, 0.1522, 0.1133],
        ...,
        [0.0311, 0.0017, 0.1044,  ..., 0.0298, 0.0286, 0.0415],
        [0.1355, 0.0715, 0.1548,  ..., 0.1365, 0.0509, 0.0523],
        [0.1938, 0.0988, 0.2327,  ..., 0.1898, 0.0829, 0.1073]],
       dtype=torch.float64)
tensor([[0.1775, 0.0901, 0.2068,  ..., 0.1699, 0.5305, 0.1723],
        [0.2378, 0.0747, 0.2506,  ..., 0.2350, 0.1423, 0.2397],
        [0.1770, 0.0

In [30]:
best_mae = 9000000000000000000
best_model_wts = deepcopy(model.state_dict())


for epoch in range(epochs):


    for phase in ['train','val']:



        if phase == 'train':

            c = 1
            running_train_mae, running_train_loss = 0, 0

            # print("In training")

            for inputs, output in train:

                if len(inputs) == batchSize:

                    # print(c)
                    c += 1

                    inputs = torch.tensor(inputs, dtype = torch.float32, requires_grad = True)
                    output = torch.reshape(torch.tensor(output, dtype = torch.float32, requires_grad = True), (batchSize,1))

                    # Forward pass
                    y_pred = model(inputs, str(epoch) + str(c))
                    loss = criterion(y_pred, output)  
                    
                    # Zero gradients, perform a backward pass, and update the weights.
                    optimizer.zero_grad()
                    grad = torch.autograd.grad(outputs = loss, inputs = inputs, retain_graph = True)
                    loss.backward()
                    optimizer.step()

                    # Update the coordinate weights
                    # https://discuss.pytorch.org/t/updatation-of-parameters-without-using-optimizer-step/34244/4
                    with torch.no_grad():
                        for name, p in model.named_parameters():
                            if name == 'SocialSig.W':
                                new_val = update_function(p, grad[0], loss, lr)
                                p.copy_(new_val)

                    running_train_mae += mae(y_pred, output).item()
                    running_train_loss += loss.item()

        if phase == 'val':

            c = 1
            running_val_mae, running_val_loss,  = 0, 0

            # print("In validation")

            for inputs, output in val:

                if len(inputs) == batchSize:

                    # print(c)
                    c += 1

                    inputs = torch.tensor(inputs, dtype = torch.float32, requires_grad = True)
                    output = torch.reshape(torch.tensor(output, dtype = torch.float32, requires_grad = True), (batchSize,1))

                    # Forward pass
                    y_pred = model(inputs, 1)
                    loss = criterion(y_pred, output)  

                    running_val_mae += mae(y_pred, output).item()
                    running_val_loss += loss.item()
                    
                    if mae(y_pred, output).item() < best_mae:
                        best_mae = mae(y_pred, output).item()
                        best_model_wts = deepcopy(model.state_dict())

                    



                    
    print("Epoch: ", epoch)  
    print("  Train:")
    print("    Loss: ", running_train_loss / c)      
    print("    MAE: ", running_train_mae / c)
    print("  Val:")
    print("    Loss: ", running_val_loss / c)      
    print("    MAE: ", running_val_mae / c)

/Users/heatherbaier/anaconda/envs/caoe/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/heatherbaier/anaconda/envs/caoe/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/heatherbaier/Desktop/CAOE/sig/socialSig.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print("    W at beginning: ", torch.tensor(self.W))
/Users/heatherbaier/Desktop/CAOE/sig/socialSig.py:56: UserWarning: To copy construct from a tensor, it is recommended 

In [31]:
model.load_state_dict(best_model_wts)

<All keys matched successfully>

In [18]:
X

array([[0.15086376, 0.01904088, 0.17205208, ..., 0.15731832, 0.13144279,
        0.09660652],
       [0.12192231, 0.04674133, 0.18605187, ..., 0.1205067 , 0.05757128,
        0.15626403],
       [0.08485822, 0.04340877, 0.10657675, ..., 0.09579383, 0.        ,
        0.05811292],
       ...,
       [0.2560983 , 0.06107871, 0.22930659, ..., 0.24920383, 0.04052688,
        0.29325165],
       [0.30418829, 0.12659953, 0.30113786, ..., 0.29673268, 0.09956838,
        0.2626708 ],
       [0.45947368, 0.26082901, 0.4582634 , ..., 0.44939452, 0.22173665,
        0.33150409]])

In [32]:
for i in range(0, 5):

    imput = torch.reshape(torch.tensor(X[i], dtype = torch.float32), (1,1,29))

    model.eval()

    print(model(imput, 1))

    W at beginning:  tensor([ 0.0114, -0.0293,  0.0205,  0.0906,  0.0087,  0.0729,  0.0042,  0.1181,
         0.0211,  0.0962,  0.0164,  0.0528,  0.0474,  0.2964,  0.1709,  0.0216,
         0.0699,  0.0503,  0.0016,  0.0387,  0.0154,  0.1112,  0.0114,  0.0112,
         0.0349,  0.0400,  0.0589,  0.0646,  0.0560])
tensor([[169.2038]], grad_fn=<AddmmBackward>)
    W at beginning:  tensor([ 0.0114, -0.0293,  0.0205,  0.0906,  0.0087,  0.0729,  0.0042,  0.1181,
         0.0211,  0.0962,  0.0164,  0.0528,  0.0474,  0.2964,  0.1709,  0.0216,
         0.0699,  0.0503,  0.0016,  0.0387,  0.0154,  0.1112,  0.0114,  0.0112,
         0.0349,  0.0400,  0.0589,  0.0646,  0.0560])
tensor([[156.2059]], grad_fn=<AddmmBackward>)
    W at beginning:  tensor([ 0.0114, -0.0293,  0.0205,  0.0906,  0.0087,  0.0729,  0.0042,  0.1181,
         0.0211,  0.0962,  0.0164,  0.0528,  0.0474,  0.2964,  0.1709,  0.0216,
         0.0699,  0.0503,  0.0016,  0.0387,  0.0154,  0.1112,  0.0114,  0.0112,
         0.0349, 

In [33]:
import pickle

In [15]:
inputs

tensor([[2.7388e-01, 3.0840e-02, 2.5688e-01, 2.7936e-01, 3.6218e-02, 1.8417e-01,
         1.2314e-01, 0.0000e+00, 0.0000e+00, 2.4442e-01, 2.6865e-01, 2.7435e-01,
         0.0000e+00, 3.8383e-02, 2.8620e-01, 1.0488e-01, 2.7305e-01, 2.4308e-01,
         1.2009e-02, 4.6390e-03, 2.4740e-01, 1.1830e-01, 1.0000e+00, 1.2805e-01,
         2.7101e-01, 1.9818e-01, 2.6964e-01, 1.1546e-01, 2.7110e-01],
        [3.8522e-02, 1.3040e-02, 4.8051e-02, 3.5860e-02, 6.5743e-03, 2.6583e-02,
         5.7025e-02, 0.0000e+00, 5.1311e-02, 4.7917e-02, 5.6560e-02, 4.9159e-02,
         0.0000e+00, 4.6879e-03, 1.3266e-01, 1.8648e-02, 6.1607e-02, 2.2079e-02,
         1.7460e-03, 2.3633e-03, 1.0237e-01, 5.4286e-03, 1.0000e+00, 2.7162e-02,
         3.8032e-02, 2.5502e-02, 3.7844e-02, 1.7950e-02, 4.9828e-02],
        [3.4198e-02, 0.0000e+00, 3.1788e-02, 3.2782e-02, 2.3091e-03, 2.3490e-02,
         0.0000e+00, 3.3332e-05, 9.1619e-05, 3.4928e-02, 8.4081e-02, 4.4449e-02,
         0.0000e+00, 4.3339e-03, 1.8147e-01, 2.036

In [26]:
test = [9.2427e-02, 3.1187e-02, 8.5081e-02, 8.8253e-02, 2.5471e-02, 6.4657e-02,
         8.7187e-02, 3.3332e-05, 0.0000e+00, 9.7802e-02, 6.6808e-02, 1.1386e-01,
         0.0000e+00, 7.2185e-03, 1.9594e-01, 4.1522e-02, 1.2835e-01, 7.3791e-02,
         2.8372e-03, 1.5616e-03, 5.2440e-02, 3.9740e-02, 1.0000e+00, 5.2138e-02,
         9.1396e-02, 9.5489e-02, 8.9893e-02, 5.8003e-02, 3.3618e-02]



print(construct_noOverlap_indices(torch.tensor(test, dtype = torch.float32), 10, 29))

tensor([[  8,  12,   7,  19,  18,  13,   4,   1,  28,  21,  15,  23,  20,  27,
           5,  10,  17,   2,   6,   3,  26,  24,   0,  25,   9,  11,  16,  14,
          22],
        [ 37,  41,  36,  48,  47,  42,  33,  30,  57,  50,  44,  52,  49,  56,
          34,  39,  46,  31,  35,  32,  55,  53,  29,  54,  38,  40,  45,  43,
          51],
        [ 66,  70,  65,  77,  76,  71,  62,  59,  86,  79,  73,  81,  78,  85,
          63,  68,  75,  60,  64,  61,  84,  82,  58,  83,  67,  69,  74,  72,
          80],
        [ 95,  99,  94, 106, 105, 100,  91,  88, 115, 108, 102, 110, 107, 114,
          92,  97, 104,  89,  93,  90, 113, 111,  87, 112,  96,  98, 103, 101,
         109],
        [124, 128, 123, 135, 134, 129, 120, 117, 144, 137, 131, 139, 136, 143,
         121, 126, 133, 118, 122, 119, 142, 140, 116, 141, 125, 127, 132, 130,
         138],
        [153, 157, 152, 164, 163, 158, 149, 146, 173, 166, 160, 168, 165, 172,
         150, 155, 162, 147, 151, 148, 171, 169, 145, 17

In [34]:
pickle.dump(model, open("./trained_weights.sav", 'wb'))

In [33]:
torch.save({
            'epoch': 50,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': criterion,
            }, "./trained_weights_nosending5.torch")

In [14]:










for t in range(epochs):
    for k in range(math.ceil(len(y)/batchSize)):
        # Prep the batch for a forward pass
        batchObs = random.sample(range(0, len(y)), batchSize)
        #batchObs = [i for i in range(0, batchSize)]
        modelX = X[batchObs]
        modelX = torch.tensor(list(modelX), requires_grad = True, dtype = torch.float32)
        modely = torch.reshape(torch.tensor(y[batchObs], dtype = torch.float32), (batchSize,1))

        # Forward pass
        y_pred = model(modelX, t)
        loss = criterion(y_pred, modely)  
        
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        grad = torch.autograd.grad(outputs=loss, inputs=modelX, retain_graph = True)
        loss.backward()
        optimizer.step()

        # Update the coordinate weights
        # https://discuss.pytorch.org/t/updatation-of-parameters-without-using-optimizer-step/34244/4
        with torch.no_grad():
            for name, p in model.named_parameters():
                if name == 'SocialSig.W':
                    new_val = update_function(p, grad[0], loss, lr)
                    p.copy_(new_val)
        

        print("Epoch: " + str(t) + " Batch: " + str(k))
        print("    Loss:     ", loss.item(), "     MAE: ", mae(y_pred, modely).item())
        print("\n")

        if loss.item() < 0:
            break



ValueError: Sample larger than population or is negative